<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/yvp_plant_functional_groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentation

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [2]:
library(tidyverse)

* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [6]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’




# Source

## Database Connection

In [7]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [8]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [9]:
billing <- bq_test_project()

## Database Query

### yvp_vegetation_cover


In [10]:
sql_functional_groups <- "SELECT *
               FROM `mpg-data-warehouse.vegetation_fixed_plot_yvp.yvp_vegetation_cover`"

In [11]:
bq_functional_groups <- bq_project_query(billing, sql_functional_groups)

In [12]:
tb_functional_groups <- bq_table_download(bq_functional_groups)

In [13]:
df_functional_groups <- as.data.frame(tb_functional_groups)

### vegetation_species_metadata

In [14]:
sql_species_metadata <- "SELECT *
                         FROM `mpg-data-warehouse.vegetation_species_metadata.vegetation_species_metadata`"

In [15]:
bq_species_metadata <- bq_project_query(billing, sql_species_metadata)

In [16]:
tb_species_metadata <- bq_table_download(bq_species_metadata)

In [17]:
df_species_metadata <- as.data.frame(tb_species_metadata)

# Wrangle

## Filter

In [18]:
# remove "NV"
df_species_metadata <- df_species_metadata %>%
  filter(key_plant_code != "NV")

In [19]:
# remove single "unknown" row
df_species_metadata <- df_species_metadata %>%
  filter(plant_life_form != "unknown")

## Recode

In [20]:
df_species_metadata %>% distinct(plant_life_cycle)

plant_life_cycle
<chr>
perennial
annual perennial
annual
unknown
annual biennial
annual biennial perennial
biennial perennial
biennial


In [21]:
df_species_metadata <- df_species_metadata %>%
  mutate(plant_life_cycle = ifelse(plant_life_cycle == "biennial perennial" |
                                   plant_life_cycle == "annual perennial" |
                                   plant_life_cycle == "annual biennial perennial" |
                                   plant_life_cycle == "annual biennial"
                                   , "multiple", plant_life_cycle))

In [22]:
df_species_metadata %>% distinct(plant_life_form)

plant_life_form
<chr>
graminoid
forb
tree
shrub
vine
